<center><img src="https://i.imgur.com/gb6B4ig.png" width="400" alt="Weights & Biases" /> </center>
<br>
<p><font size="3" face="Arial" font-size="large"><ul type="square">
    
<li><a href="#c1">🔑 Ключевые особенности  </a></li>

<li><a href="#c2">📲 Регистрация учетной записи и вход в систему </a></li>
<li><a href="#с3">✍️ Логируем реализацию MLP из урока про нейросети</a>
<li><a href="#c4">💎 Создание артефакта W&B</a>
<li><a href="#c5">📝💥 Теперь залогируем бустинг </a>
<li><a href="#c6">❄️ Полезные ссылки </a>

</li></ul></font></p>

# Да кто такой, этот ваш `W&B`?

`Weights & Biases (W&B)` - это многофункциональная платформа для отслеживания экспериментов, логирования результатов, которая позволяет облегчить и ускорить рутинные процессы при обучении моделей.

# 🔑 Ключевые особенности
<div class="alert alert-info">

* Позволяет отслеживать показатели работы модели в режиме реального времени и сразу же выявлять проблемные места.
* Визуализация результатов - графики, изображения, видео, аудио, 3D-объекты и многое [другое](https://docs.wandb.ai/guides/track/log#logging-objects).
* Позволяет тренировать модели с разных устройств и хранить результаты в одном месте, также удобно при командной работе над задачей
* Использование артефактов `W&B` позволяет отслеживать и создавать версии ваших наборов данных, моделей, зависимостей и результатов в конвейерах машинного обучения.
* Низкий порог входа (можно начать с 6 строк кода), также есть готовые интеграции со всеми популярными DS фреймворками

![centralized dashboard](https://i.imgur.com/BGgfZj3.png)

## Установка `wandb`

`wandb` (библиотека `W&B`), по умолчанию установлена на `Kagglе`

Поскольку образ `Kaggle` не часто обновляется, а `wandb` постоянно выпускает новые версии, рекомендуется установить последнюю версию с помощью флага `--upgrade`.

In [1]:
#!pip install --upgrade -q wandb

## 📲 Регистрация учетной записи и вход в систему
<div class="alert alert-info">

Вам понадобится уникальный ключ API для входа в `Weights & Biases`.

1. Если у вас нет учетной записи `Weights & Biases`, вы можете перейти на https://wandb.ai/site и создать БЕСПЛАТНУЮ учетную запись.
2. Доступ к ключу API: https://wandb.ai/authorize.

<div class="alert alert-info">

На `Kaggle` можно авторизоваться в `W&B` двумя способами:

1. С помощью `wandb.login ()`. Он запросит ключ API, который вы можете скопировать + вставить.
2. Используя `Kaggle secrets` для хранения ключа API и использовать приведенный ниже фрагмент кода для входа в систему. Прочтите это [обсуждение](https://www.kaggle.com/product-feedback/114053), чтобы узнать больше о Kaggle secrets.

<div class="alert alert-info">


```python
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
wandb_api = user_secrets.get_secret("wandb_api") 
wandb.login(key=wandb_api)
```
[Подробнее о входе в W&B](https://docs.wandb.ai/ref/cli/wandb-login).

In [1]:
import wandb

wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /home/ivanich_spb/.netrc
wandb: ERROR Unable to read ~/.netrc


True

## ✍️ Залогируем, для примера, реализацию `MLP` из урока про нейросети.

In [3]:
import torch
import os
import copy
import sys
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [4]:
# Загружаем train-датасет, который мы сохранили на шаге quickstart
rides_info = pd.read_csv('../data/quickstart_train.csv')
rides_info.head(3)

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class,mean_rating,distance_sum,rating_min,speed_max,user_ride_quality_median,deviation_normal_count,user_uniq
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,109.99,another_bug,4.737759,1.214131e+07,0.1,180.855726,0.023174,174,170
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,34.48,electro_bug,4.480517,1.803909e+07,0.0,187.862734,12.306011,174,174
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,34.93,gear_stick,4.768391,1.588366e+07,0.1,102.382857,2.513319,174,173


In [5]:
drop_cols = ['car_id', 'target_reg', 'target_class']
cat_cols = ['car_type', 'fuel_type', 'model']

In [6]:
# закодируем категориальные фичи в one hot encoding вектора
rides_info = pd.get_dummies(rides_info, columns=cat_cols)

# заполним пропущенные значения медианным значением по столбцу
rides_info.fillna(rides_info.median(numeric_only=True), inplace=True)

In [7]:
# переведем строковые значения категориального таргета в целочисленные
le = LabelEncoder()
rides_info['target_class'] = le.fit_transform(rides_info['target_class'])

In [8]:
# числовые переменные, которые подвергнем трансформации
num_cols = [col for col in list(rides_info.columns) if col not in drop_cols]

In [9]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

df = scaler.fit_transform(rides_info[num_cols])
df = pd.DataFrame(df, columns=num_cols)

target_scaler = RobustScaler()
target = target_scaler.fit_transform(rides_info['target_reg'].values.reshape(-1, 1))

df['target_reg'] = target
df['target_class'] = rides_info['target_class']

In [10]:
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [11]:
# ВАЖНО! - фиксируем воспроизводимость
def seed_everything(seed=42):
    
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [12]:
# для наших данных и размера нейросети подойдет запуск на cpu
device = torch.device('cpu')

In [13]:
train, test = train_test_split(df, test_size=0.2, random_state=42)

### Сохраняем гиперпараметры
Сохраняем гиперпараметры в виде словаря, для того чтобы позже зарегистрировать этот конфигурационный файл в W&B.

In [14]:
# Определяем гиперпараметры

CONFIG = dict (
    hidden_size=128,
    dropout=0.1,
    lr=1e-3,
    batch_size=128,
    num_workers=os.cpu_count(),
    epochs=20,
    num_features=train.shape[1]-2, # количество фичей, подаваемое на вход
    num_tar_2=train.target_class.nunique(), # количество выходов равно количеству предсказываемых классов
    architecture = "MLP",
    infra = "Kaggle"
)

##  Строим входной конвейер

In [15]:
# датасет выдает фичи и значения целевых переменных
class Rides(Dataset):
    
    def __init__(self, df):
        self.df = df
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx,:]
        
        data = row.drop(labels=['target_reg', 'target_class'])
        
        data = torch.FloatTensor(data.values.astype('float'))
        tar_1 = torch.tensor(row['target_reg']).float()
        tar_2 = row['target_class'].astype('int')
        
        return data, tar_1, tar_2

In [16]:
# Build data loaders

train_datasets = {'train': Rides(train),
                  'val': Rides(test)}

dataloaders = {x: torch.utils.data.DataLoader(train_datasets[x], 
                                                   batch_size=CONFIG['batch_size'], 
                                                   shuffle=True, 
                                                   num_workers=CONFIG['num_workers'])
                    for x in ['train', 'val']}

## Настраиваем модель - Класс `TabularNN`

In [17]:
# Построим архитектуру mlp с двумя головами для регрессии и классификации

class TabularNN(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.mlp = nn.Sequential(
                          nn.Linear(cfg['num_features'], cfg['hidden_size']),
                          nn.Dropout(cfg['dropout']),
                          nn.ReLU(),
                          nn.Linear(cfg['hidden_size'], cfg['hidden_size']),
                          nn.Dropout(cfg['dropout']),
                          nn.ReLU(),
                          nn.Linear(cfg['hidden_size'], cfg['hidden_size'] // 2),
                          )
        
        self.regressor = nn.Sequential(
            nn.Linear(cfg['hidden_size'] // 2, 1)
        )
        self.classifier = nn.Sequential(
            nn.Linear(cfg['hidden_size'] // 2, cfg['num_tar_2'])
        )

    def forward(self, data):
        x = self.mlp(data)
        tar1 = self.regressor(x)
        tar2 = self.classifier(x)
        return tar1.view(-1), tar2

## Обучаем и логируем результаты в `W&B`
<div class="alert alert-info">

В этом разделе будем использовать:

* [`wandb.init()`](https://docs.wandb.ai/guides/track/launch): инициализировать новый запуск.
* [`wandb.finish()`](https://docs.wandb.ai/ref/python/finish):  завершить и закрыть пробег.
* [`wandb.config`](https://docs.wandb.ai/guides/track/config): объект, в котором хранятся гиперпараметры и настройки, связанные с запуском.


`Run` (или [объект `wandb.Run`](https://docs.wandb.ai/ref/python/run)) - это единица вычисления `W&B`, обычно это 1 эксперимент обучения модели.

In [18]:
# Инициализируем модель
model = TabularNN(CONFIG).to(device)

# оптимайзер и лоссы для регрессии и классификации
optimizer = torch.optim.Adam(model.parameters(), lr = CONFIG['lr'])
regression_criterion = nn.MSELoss().to(device)
classification_criterion = nn.CrossEntropyLoss().to(device)

## `wandb.init()` для инициализации нового прогона `W&B`.

В `pipeline` обучения модели вы можете добавить `wandb.init ()` в начало обучающего сценария, а также в инференс, и каждая часть будет отслеживаться как запуск в `W&B`.

In [19]:
# Добавим в CONFIG имя модели
CONFIG['model_name'] = 'Tabular_NN'
print('Training configuration: ', CONFIG)

# Initialize W&B run
run = wandb.init(project='Course contest',
                 config=CONFIG,
                 group='MLP', 
                 job_type='train')


wandb: Currently logged in as: ivanich. Use `wandb login --relogin` to force relogin


Training configuration:  {'hidden_size': 128, 'dropout': 0.1, 'lr': 0.001, 'batch_size': 128, 'num_workers': 20, 'epochs': 20, 'num_features': 43, 'num_tar_2': 9, 'architecture': 'MLP', 'infra': 'Kaggle', 'model_name': 'Tabular_NN'}


<div class="alert alert-info">

Ипользуемые аргументы `wandb.init ()`:

* `project`: этот аргумент указывает имя проекта `W&B`, в который отправляется запуск. Создаем на платформе `W&B` новый проект с названием «Course_contest» и одновременно отправляем в него run.
* `config`: этот аргумент устанавливает `wandb.config`, объект, подобный словарю, в котором хранятся гиперпараметры, параметры ввода и другие независимые переменные.
* `group`: этот аргумент указывает значение для группировки отдельных прогонов, чтобы было легче сравнивать прогоны из разных архитектур.
* `job_type`: этот аргумент указывает тип выполнения, например, «`train`» или «`evaluate`». Установка типа `run` упрощает последующую фильтрацию и группировку `run`, например, чтобы сравнить несколько «`train`» `run`-ов.

## Обновляем `wandb.config` 
<div class="alert alert-info">

Сохранение конфигурации тренировки полезно для анализа экспериментов и воспроизведения вашей работы позже. С помощью `W&B` вы также можете группировать `run`-ы по значениям конфигурации, что означает, что вы можете сравнивать настройки разных `run`-ов и видеть, как они влияют на результат.

Есть несколько способов настроить `wandb.config`:

* Задать `wandb.config` с аргументом `wandb.init (config)`, как указано выше.
* Установить `wandb.config` напрямую.
* См. Дополнительные параметры настройки в этом [Colab](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/wandb-log/Configs_in_W%26B.ipynb#scrollTo=xFf3zjBSixC1).

In [20]:
# Добавляем "type" and "kaggle_competition" to `wandb.config` directly
wandb.config.type = 'baseline'
wandb.config.kaggle_competition = 'Competitive Data Science Course Contest'

# Close W&B run
run.finish()

### Проведем серию из 5 экспериментов для подбора параметра `dropout rate`

In [21]:
import random

num_epochs = CONFIG['epochs']
verbose = 5
for _ in range(5):
    
    # Изменим CONFIG - определим dropout как рандомную величину в заданном диапазоне значений
    CONFIG['dropout'] = random.uniform(0.01, 0.80)
    model = TabularNN(CONFIG).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr = CONFIG['lr'])
    
    # 🐝 initialize a wandb run
    wandb.init(project='Course contest',
               config=CONFIG,
               group='MLP', 
               job_type='train'
            )
    
    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = np.inf

    for epoch in range(num_epochs):
        if epoch % verbose == 0:
            print('Epoch {}/{}'.format(epoch, num_epochs - 1))
            print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0

            # Iterate over data.
            for inputs, labels_1, labels_2 in dataloaders[phase]:
                inputs = inputs.to(device)
                labels_1 = labels_1.to(device)
                labels_2 = labels_2.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss

                    outputs_1, outputs_2 = model(inputs)
                    loss_1 = regression_criterion(outputs_1, labels_1)
                    loss_2 = classification_criterion(outputs_2, labels_2)

                    loss = loss_1 + loss_2

                    _, preds_2 = torch.max(outputs_2, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                val_acc_history.append(running_loss)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            if epoch % verbose == 0:
                print('{} Loss: {:.4f}'.format(phase, epoch_loss))
            
            # 🐝 Log train and validation metrics to wandb
            wandb.log({'{} loss'.format(phase): epoch_loss, 'dropout':CONFIG['dropout']})

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_model_wts = copy.deepcopy(model.state_dict())
                best_loss = epoch_loss


    # load best model weights
    model.load_state_dict(best_model_wts)
    
    # 🐝 Close your wandb run 
    wandb.finish()

Epoch 0/19
----------
train Loss: 2.7689
val Loss: 2.7900
Epoch 5/19
----------
train Loss: 2.4713
val Loss: 2.3244
Epoch 10/19
----------
train Loss: 1.9001
val Loss: 1.6707
Epoch 15/19
----------
train Loss: 1.6445
val Loss: 1.4354


dropout,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train loss,███▇▇▆▅▅▄▄▃▃▂▂▂▂▁▁▁▁
val loss,███▇▇▆▅▄▃▃▃▂▂▂▂▁▁▁▁▁
dropout,0.75141
train loss,1.5537
val loss,1.33253


Epoch 0/19
----------
train Loss: 2.7286
val Loss: 2.7112
Epoch 5/19
----------
train Loss: 1.4087
val Loss: 1.4504
Epoch 10/19
----------
train Loss: 1.0415
val Loss: 1.1928
Epoch 15/19
----------
train Loss: 0.9289
val Loss: 1.1564


dropout,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train loss,█▇▆▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁
val loss,█▇▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
dropout,0.02856
train loss,0.89752
val loss,1.15334


Epoch 0/19
----------
train Loss: 2.7433
val Loss: 2.7235
Epoch 5/19
----------
train Loss: 1.4568
val Loss: 1.4543
Epoch 10/19
----------
train Loss: 1.1334
val Loss: 1.2119
Epoch 15/19
----------
train Loss: 1.0169
val Loss: 1.1470


dropout,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train loss,█▇▆▅▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁
val loss,█▇▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
dropout,0.15102
train loss,0.9907
val loss,1.18091


Epoch 0/19
----------
train Loss: 2.7211
val Loss: 2.6880
Epoch 5/19
----------
train Loss: 1.3915
val Loss: 1.4237
Epoch 10/19
----------
train Loss: 1.0552
val Loss: 1.1858
Epoch 15/19
----------
train Loss: 0.9562
val Loss: 1.1715


dropout,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train loss,█▇▆▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁
val loss,█▇▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
dropout,0.03408
train loss,0.89437
val loss,1.1524


Epoch 0/19
----------
train Loss: 2.7569
val Loss: 2.7505
Epoch 5/19
----------
train Loss: 1.8299
val Loss: 1.6234
Epoch 10/19
----------
train Loss: 1.3681
val Loss: 1.2868
Epoch 15/19
----------
train Loss: 1.2522
val Loss: 1.2210


dropout,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train loss,██▇▆▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁
val loss,██▇▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁
dropout,0.49939
train loss,1.19739
val loss,1.20713


Используйте `run.finish ()`, чтобы закрыть инициализированный `run W&B` после завершения `job_type`.

Каждый `run` получает свою собственную страницу, на вкладках которой содержится дополнительная информация о запуске.

## 💎 Создание артефакта `W&B`.
<div class="alert alert-info">

`W&B Artifacts` позволяет вам регистрировать данные, которые входят (например, набор данных) и выходят (например, веса обученной модели) из этих процессов.

Другими словами, артефакты - это способ сохранить ваши наборы данных и модели. Вы можете использовать [этот Colab](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/wandb-artifacts/Pipeline_Versioning_with_W%26B_Artifacts.ipynb), чтобы узнать больше об артефактах.

### Сохраняем свою работу с помощью `wandb.log_artifact ()` 

В `run` есть три шага для создания и сохранения артефакта модели.

1. Создайте пустой артефакт с помощью `wandb.Artifact ()`.
2. Добавьте файл модели в Артефакт с помощью `wandb.add_file ()`.
3. Вызовите `wandb.log_artifact ()`, чтобы сохранить Артефакт.

In [22]:
# Save model
torch.save(model.state_dict(), 'tab_model.pth')

# Initialize a new W&B run
run = wandb.init(project='Course contest',
                 config=CONFIG,
                 group='MLP', 
                 job_type='save') # Note the job_type

# Update `wandb.config`
wandb.config.type = 'baseline'
wandb.config.kaggle_competition = 'Competitive Data Science Course'

# Save model as Model Artifact
artifact = wandb.Artifact(name='best_tab_NN', type='model') # Задаем произвольное имя
artifact.add_file('tab_model.pth')
run.log_artifact(artifact)

# Finish W&B run
run.finish()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


# <center> 📝💥 Теперь залогируем бустинг <center>

In [26]:
#!pip install lightgbm -q

In [23]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split

In [24]:
run = wandb.init(project='Course contest',
                 job_type='train-model',
                 group='LGBM',
                 config={'wandb_nb':'wandb_course_contest'})  # config is optional here

In [26]:
run.config.update(bst_params)
run.config.update({'early_stopping_rounds': 40})

In [27]:
X = df.drop(['target_reg','target_class'], axis=1)
y = df['target_class']

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
bst_params = {
        'learning_rate': 0.1,
        'min_child_weight': 100,
        'n_estimators': 125,
        'random_state': 42,
        'reg_alpha': 0,
        'reg_lambda': 0,
        'early_stopping_round':run.config['early_stopping_rounds'],
        'verbosity':-1,
    }

<div class="alert alert-info">

В `wandb` есть встроенная поддержка библиотеки `LightGBM` через `callback`

In [30]:
from wandb.integration.lightgbm import wandb_callback, log_summary
import lightgbm as lgb

lgbmodel = lgb.LGBMClassifier(**bst_params)

# Train the model, using the wandb_callback for logging
lgbmodel.fit(X_train, y_train, eval_set=[(X_test, y_test)], 
             callbacks=[wandb_callback()])

[10]	valid_0's multi_logloss: 1.95486
[20]	valid_0's multi_logloss: 1.71644
[30]	valid_0's multi_logloss: 1.61596
[40]	valid_0's multi_logloss: 1.57421
[50]	valid_0's multi_logloss: 1.56006
[60]	valid_0's multi_logloss: 1.55172
[70]	valid_0's multi_logloss: 1.55144
[80]	valid_0's multi_logloss: 1.55144
[90]	valid_0's multi_logloss: 1.55144
[100]	valid_0's multi_logloss: 1.55144


/home/ivanich_spb/.local/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/ivanich_spb/.local/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


LGBMClassifier(min_child_weight=100, n_estimators=125, random_state=42,
               reg_alpha=0, reg_lambda=0)

In [31]:
# Save the booster to disk
model_name = f'{run.name}_model.mod'
bstr = lgbmodel.booster_
bstr.save_model(model_name)

# Get the booster's config
config = lgbmodel.get_params()

# Log the trained model to W&B Artifacts, including the booster's config
model_art = wandb.Artifact(name=model_name, type='model', metadata=config)
model_art.add_file(model_name)
run.log_artifact(model_art);

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [32]:
# Log booster metrics
run.summary["best_score"] = bstr.best_score
run.summary["best_iteration"] = bstr.best_iteration

# Log validation metrics
preds = lgbmodel.predict(X_test)
run.summary["f1_score"] = f1_score(y_test, preds, average='macro')
run.summary["accuracy"] = accuracy_score(y_test, preds)

#### Finish the W&B Run

In [33]:
run.finish()

iteration,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
valid_0_multi_logloss,█▇▆▅▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.55769
best_iteration,62
f1_score,0.51252
iteration,101


# ❄️ Полезные ссылки:

<div class="alert alert-info">

Вот несколько релевантных ссылок по теме:


* Ознакомьтесь с [официальной документацией](https://docs.wandb.ai/), чтобы узнать больше о передовых методах работы и дополнительных функциях.

* Ознакомьтесь с [репозиторием GitHub с примерами](https://github.com/wandb/examples), где вы найдете тщательно отобранные и минимальные примеры. Это может быть хорошей отправной точкой.

    

# Примеры Kaggle ноутбуков с Weights & Biases.

* [EfficientNet+Mixup+K-Fold using TF and wandb](https://www.kaggle.com/ayuraj/efficientnet-mixup-k-fold-using-tf-and-wandb)
* [HPA: Segmentation Mask Visualization with W&B](https://www.kaggle.com/ayuraj/hpa-segmentation-mask-visualization-with-w-b)
* [HPA: Multi-Label Classification with TF and W&B](https://www.kaggle.com/ayuraj/hpa-multi-label-classification-with-tf-and-w-b)
* [🐦BirdCLEF: Quick EDA with W&B](https://www.kaggle.com/ayuraj/birdclef-quick-eda-with-w-b)